In [3]:
import os
import time
import numpy as np
import pandas as pd
import pytz
from measurement_stat import MEASUREMENT_SOURCE_VALUE_STATS
from datetime import datetime, timedelta, time as datetime_time, timezone
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler


VALUE_MAP = ['HR','RR','SpO2','Pulse','Temp','ABPm','ABPd','ABPs','NBPm','NBPs','NBPd','SPO2-%','SPO2-R',
'Resp','PVC','ST-II','etCO2','SpO2 r','imCO2','ST-V1','ST-I','ST-III','ST-aVF','ST-aVL','ST-aVR',
'awRR','CVPm','AoM','ST-V2','ST-V3','ST-V4','ST-V5','ST-V6','SpO2T','T1','TV','Cdyn','PEEP','RRaw',
'TVin','inO2','AoD','AoS','InsTi','MINVOL','MnAwP','PIP','MVin','PB','Poccl','Pplat',
'MV','Patm','Ppeak','Rinsp','ST-V','sInsTi','sPEEP','sTV','sTrig','sPSV','Rexp','highP',
'sAPkFl','sAWRR','sFIO2','sPIF','sMV','sO2','sRisTi','ARTd','ARTm','ARTs','PAPm','sSIMV']

MEASUREMENT_NORMALIZATION = ['mean', 'predefined']


In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #"last expr -> all로 바꾸면 전체가 나온다. "
pd.options.display.max_columns = 200
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 100

In [2]:
data_path='../../data/train'
common_path='../../data/volume'

task_path='../../data/volume/local_test'
# os.mkdir(task_path)
is_train=True

group_hour=1
timestep_per_data=128

measurement_normalize='mean'

condition_min_limit=0
condition_group=False

valid_size=0.2
data_split_random_seed=1235
pytest=False

In [3]:
  def extract_outcome_cohort():
    start_time = time.time()
    cohort_df = pd.read_csv(os.path.join(data_path, 'OUTCOME_COHORT.csv'), encoding='windows-1252')

    cohort_df.COHORT_START_DATE = pd.to_datetime(cohort_df.COHORT_START_DATE)
    cohort_df.COHORT_END_DATE = pd.to_datetime(cohort_df.COHORT_END_DATE)
    print("data_loader extract_outcome_cohort time:", time.time() - start_time)
    return cohort_df

  def extract_person():
    start_time = time.time()
    person_df = pd.read_csv(os.path.join(data_path, 'PERSON_NICU.csv'), encoding='windows-1252')
    person_df = pd.concat([
        person_df[['PERSON_ID', 'BIRTH_DATETIME']],
        pd.get_dummies(person_df.GENDER_SOURCE_VALUE, prefix='gender')
    ], axis=1)

    # 생일 컬럼 타입 설정
    person_df.BIRTH_DATETIME = pd.to_datetime(person_df.BIRTH_DATETIME, utc=True)
    # 여성/남성 컬럼 1개로 처리
    person_df.rename(columns={'gender_M': 'GENDER'}, inplace=True)
    if 'gender_F' in person_df.columns:
      del person_df['gender_F']

    print("data_loader extract_person time:", time.time() - start_time)
    return person_df

  def extract_condition():
    start_time = time.time()
    condition_df = pd.read_csv(os.path.join(data_path, 'CONDITION_OCCURRENCE_NICU.csv'), encoding='windows-1252',
                               usecols=['PERSON_ID', 'CONDITION_SOURCE_VALUE', 'CONDITION_START_DATETIME'])
    # Null 이거나 값이 빈 것을 날림
    condition_df = condition_df[pd.notnull(condition_df.CONDITION_SOURCE_VALUE)]
    condition_df = condition_df[condition_df.CONDITION_SOURCE_VALUE.str.len() > 0]

    if condition_group:
      condition_df.CONDITION_SOURCE_VALUE = condition_df.CONDITION_SOURCE_VALUE.str.slice(stop=3)

    # 컬럼 타입 설정
    condition_df.CONDITION_START_DATETIME = pd.to_datetime(condition_df.CONDITION_START_DATETIME, utc=True)

    print("data_loader extract_condition time:", time.time() - start_time)
    return condition_df

  def extract_measurement():
    start_time = time.time()
    measurement_df = pd.read_csv(os.path.join(data_path, 'MEASUREMENT_NICU.csv'), 
                                 encoding='windows-1252',
                                 usecols=['PERSON_ID', 'MEASUREMENT_DATETIME',
                                          'MEASUREMENT_SOURCE_VALUE', 'VALUE_AS_NUMBER']
                                 )
#     if measurement_normalize == MEASUREMENT_NORMALIZATION[0]:
#       # source_value 맵핑
#       source_value_invert_map = {}
#       for new_value in MEASUREMENT_SOURCE_VALUE_MAP:
#         for table_value in MEASUREMENT_SOURCE_VALUE_MAP[new_value]:
#           source_value_invert_map[table_value] = new_value
#       measurement_df.MEASUREMENT_SOURCE_VALUE = measurement_df.MEASUREMENT_SOURCE_VALUE.replace(source_value_invert_map)

      # 맵핑이된 정보만 남긴다
    measurement_df = measurement_df[measurement_df.MEASUREMENT_SOURCE_VALUE.isin(VALUE_MAP)]

    # 컬럼 타입 설정
    measurement_df.MEASUREMENT_DATETIME = pd.to_datetime(measurement_df.MEASUREMENT_DATETIME, utc=True)

    # source_value별 평균값 추출
    if is_train:
      measurement_mean_df = measurement_df.groupby('MEASUREMENT_SOURCE_VALUE').VALUE_AS_NUMBER.mean()
      measurement_mean_df.to_pickle(os.path.join(common_path, 'measurement_mean.pkl'))
    else:
      # inference일 경우 저장된 걸 불러온다
      measurement_mean_df = pd.read_pickle(os.path.join(common_path, 'measurement_mean.pkl'))

    print("data_loader extract_measurement time:", time.time() - start_time)
    return measurement_df, measurement_mean_df
cohort_df = extract_outcome_cohort()
person_df = extract_person()
condition_df = extract_condition()
measurement_df,measurement_mean_df = extract_measurement()

data_loader extract_outcome_cohort time: 0.018996000289916992
data_loader extract_person time: 0.010002374649047852
data_loader extract_condition time: 0.009009838104248047
data_loader extract_measurement time: 2.6911568641662598


In [4]:
  def groupby_hour_condition( condition_df):
    start_time = time.time()

    condition_df['CONDITION_DATE'] = condition_df.CONDITION_START_DATETIME.dt.date
    condition_df['CONDITION_DATE'] = pd.to_datetime(condition_df.CONDITION_DATE, utc=True)

    if is_train and condition_min_limit > 0:
      condition_group = condition_df.groupby('CONDITION_SOURCE_VALUE').PERSON_ID.count()
      condition_group = condition_group[condition_group > condition_min_limit].index

      condition_df = condition_df[condition_df.CONDITION_SOURCE_VALUE.isin(condition_group)]

    # 진단은 시간이 없다. 당일의 마지막에 진단 받은걸로 가정한다
    condition_df['HOURGRP'] = 23 // group_hour

    group_cols = ['PERSON_ID', 'CONDITION_DATE', 'HOURGRP', 'CONDITION_SOURCE_VALUE']

    condition_df['DUMMY'] = condition_df['CONDITION_SOURCE_VALUE']
    condition_df = condition_df.groupby(group_cols) \
        .DUMMY.count().unstack().reset_index().fillna(0)

    condition_df = condition_df.rename(columns={'CONDITION_DATE': 'DATE'})

    condition_col_filename = os.path.join(task_path, 'condition_cols.npy')
    if is_train:
      # 컬럼 이름 저장
      np.save(condition_col_filename, np.array(condition_df.columns))
    else:
      # 컬럼 로드
      condition_cols = np.load(condition_col_filename, allow_pickle=True)
      new_condition_list = []
      for col in condition_cols:
        if col in condition_df.columns:
          new_condition_list.append(condition_df[col])
        else:
          new_condition_list.append(pd.Series([0] * condition_df.shape[0]))

      condition_df = pd.concat(new_condition_list, axis=1)
      condition_df.columns = condition_cols
    print("data_loader groupby_hour_condition time:", time.time() - start_time)
    return condition_df

In [5]:
condition_df = groupby_hour_condition(condition_df)

data_loader groupby_hour_condition time: 0.018995285034179688


In [6]:
  def _clip_measurement(measurement_source_value, value_as_number):
    if value_as_number > MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['95%']:
      value_as_number = MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['95%']
    elif value_as_number < MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['5%']:
      value_as_number = MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['5%']
    return value_as_number

  def groupby_hour_measurement(measurement_df):
    start_time = time.time()
    # timestamp로 join 하기 위하여 시간 포맷을 utc로 통일
    measurement_df['MEASUREMENT_DATE'] = measurement_df.MEASUREMENT_DATETIME.dt.date
    measurement_df['MEASUREMENT_DATE'] = pd.to_datetime(measurement_df.MEASUREMENT_DATE, utc=True)

    measurement_df['MEASUREMENT_HOUR'] = measurement_df.MEASUREMENT_DATETIME.dt.hour
    measurement_df['MEASUREMENT_HOURGRP'] = measurement_df.MEASUREMENT_HOUR // group_hour

    # 평균값 이용하여 Normalize
    if measurement_normalize == MEASUREMENT_NORMALIZATION[0]:
      measurement_df = pd.merge(measurement_df,
                                measurement_mean_df.reset_index().rename(
                                    columns={'VALUE_AS_NUMBER': 'MEAN_VALUE'}),
                                on='MEASUREMENT_SOURCE_VALUE', how='left')
      measurement_df.VALUE_AS_NUMBER = measurement_df.VALUE_AS_NUMBER / measurement_df.MEAN_VALUE
    # 생체신호 범위를 이용하여 Normalize
    elif measurement_normalize == MEASUREMENT_NORMALIZATION[1]:
      measurement_df.VALUE_AS_NUMBER = measurement_df.apply(lambda row:
                                                            _clip_measurement(
                                                                row['MEASUREMENT_SOURCE_VALUE'],
                                                                row['VALUE_AS_NUMBER']),
                                                            axis=1)

      # TODO
    group_cols = ['PERSON_ID', 'MEASUREMENT_DATE', 'MEASUREMENT_HOURGRP', 'MEASUREMENT_SOURCE_VALUE']
    agg_list = ['count', 'min', 'max', 'mean', 'std', 'var']
    measurement_df['VALUE_DIFF'] = measurement_df.groupby(group_cols).VALUE_AS_NUMBER.diff()

    measurement_diff_df = pd.pivot_table(measurement_df, 
                                         values='VALUE_DIFF', index=group_cols[:-1],
                                         columns='MEASUREMENT_SOURCE_VALUE', aggfunc=['mean','max','min'])

    measurement_diff_df.columns = [('diff', '{}_{}'.format(v[0],v[1])) for v in measurement_diff_df.columns]

    measurement_df = measurement_df.groupby(group_cols).VALUE_AS_NUMBER.agg(agg_list).fillna(0).unstack().fillna(method='ffill').fillna(method='bfill')

    measurement_df = pd.concat([measurement_df, measurement_diff_df], axis=1).reset_index()

    if measurement_df.isnull().sum().sum() >0:
        print("there is Na after interpolation")
        measurement_df = measurement_df.fillna(0)
        
    # 사용한 후 삭제
    del measurement_diff_df
    # 컬럼 이름 정제 (그룹화 하기 쉽게)
    new_cols = []
    for col in measurement_df.columns:
      
      if col[1] == '':
        new_cols.append(col[0])
      elif col[0] in agg_list + ['diff']:
        new_cols.append((col[1], col[0]))
    measurement_df.columns = new_cols

#     #minmax scale
#     scaler = MinMaxScaler(feature_range=(-1,1))
#     scaler = scaler.fit(measurement_df.iloc[:,3:])
#     measurement_df.iloc[:,3:] = scaler.transform(measurement_df.iloc[:,3:])
    
    measurement_df = measurement_df.rename(columns={'MEASUREMENT_DATE': 'DATE',
                                                    'MEASUREMENT_HOURGRP': 'HOURGRP'})

    measurement_col_filename = os.path.join(task_path, 'measurement_cols.npy')
    if is_train:
      # 컬럼 이름 저장
      np.save(measurement_col_filename, np.array(measurement_df.columns))
    else:
      # 컬럼 로드
      measurement_cols = np.load(measurement_col_filename, allow_pickle=True)
      new_measurement_list = []
      for col in measurement_cols:
        if col in measurement_df.columns:
          new_measurement_list.append(measurement_df[col])
        else:
          new_measurement_list.append(pd.Series([0] * measurement_df.shape[0]))

      measurement_df = pd.concat(new_measurement_list, axis=1)
      measurement_df.columns = measurement_cols
    print("data_loader groupby_hour_measurement time:", time.time() - start_time)
    return measurement_df

In [7]:
measurement_df = groupby_hour_measurement(measurement_df)

there is Na after interpolation
data_loader groupby_hour_measurement time: 7.178995132446289


### Autoencoder 작업중 

In [1]:
from data_loader import DataLoader
import os
task_path='../../data/volume/local_test'
data_loader = DataLoader(data_path=os.path.join('../../data', 'train'),
                         common_path=os.path.join("../../data", 'volume'),
                         measurement_normalize='mean',
                         is_train = True,
                         task_path=task_path)

C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.

Load files {'person': 'PERSON_NICU.csv', 'condition': 'CONDITION_OCCURRENCE_NICU.csv', 'measurement': 'MEASUREMENT_NICU.csv', 'outcome': 'OUTCOME_COHORT.csv'}
data_loader extract_outcome_cohort time: 0.014000177383422852
data_loader extract_person time: 0.006001949310302734
data_loader extract_condition time: 0.00599980354309082
data_loader extract_measurement time: 8.418001413345337
data_loader groupby_hour_condition time: 0.011002302169799805
data_loader groupby_hour_measurement time: 14.459994792938232
data_loader make_person_sequence time: 0.13303828239440918
X (2592,)
Y (2592,)
Key (2592, 2)
data_loader make_data time: 0.3940012454986572
data_loader split_data time: 0.09100055694580078


In [6]:
data_loader.mid_term[["count"]]

count                                               \
MEASUREMENT_SOURCE_VALUE     BT  DBP   IDBP   IMBP   ISBP  MBP     PR     RR   
0                           0.0  0.0    0.0    0.0    0.0  0.0  142.0   30.0   
1                           0.0  0.0   38.0   38.0   38.0  0.0  239.0  120.0   
2                           0.0  0.0  117.0  117.0  117.0  0.0  240.0  120.0   
3                           0.0  0.0  120.0  120.0  120.0  0.0  240.0  120.0   
4                           0.0  0.0  112.0  116.0  112.0  0.0  240.0  120.0   
...                         ...  ...    ...    ...    ...  ...    ...    ...   
5256                      240.0  2.0    0.0    0.0    0.0  2.0  480.0  240.0   
5257                      240.0  4.0    0.0    0.0    0.0  4.0  452.0  240.0   
5258                      240.0  0.0    0.0    0.0    0.0  2.0  336.0  240.0   
5259                      240.0  0.0    0.0    0.0    0.0  0.0  354.0  240.0   
5260                       86.0  0.0    0.0    0.0    0.0  0.0  106.0   86.0   

                                      
MEASUREMENT_SOURCE_VALUE  SBP   SPO2  
0                         0.0  112.0  
1                         0.0  120.0  
2                         0.0  120.0  
3                         0.0  120.0  
4                         0.0  120.0  
...                       ...    ...  
5256                      2.0  240.0  
5257                      4.0  212.0  
5258                      0.0   96.0  
5259                      0.0  114.0  
5260                      0.0   20.0  

[5261 rows x 10 columns]

In [3]:
data_loader.post_term

,PERSON_ID,DATE,HOURGRP,"(BT, count)","(DBP, count)","(IDBP, count)","(IMBP, count)","(ISBP, count)","(MBP, count)","(PR, count)",...,"(BT, diff)","(DBP, diff)","(IDBP, diff)","(IMBP, diff)","(ISBP, diff)","(MBP, diff)","(PR, diff)","(RR, diff)","(SBP, diff)","(SPO2, diff)"
0,1.000000e+01,2036-04-11 00:00:00+00:00,10,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-0.420168,...,0.071038,0.033333,0.450980,-0.280081,0.578947,0.152542,-0.140250,0.850782,-0.103448,0.714557
1,1.000000e+01,2036-04-11 00:00:00+00:00,11,-1.000000,-1.0,-0.366667,-0.366667,-0.366667,-1.0,-0.012605,...,0.071038,0.033333,1.000000,-0.131630,0.294452,0.152542,-0.088972,0.842784,-0.103448,0.670057
2,1.000000e+01,2036-04-11 00:00:00+00:00,12,-1.000000,-1.0,0.950000,0.950000,0.950000,-1.0,-0.008403,...,0.071038,0.033333,0.350913,-0.303757,0.651543,0.152542,-0.088660,0.840206,-0.103448,0.662981
3,1.000000e+01,2036-04-11 00:00:00+00:00,13,-1.000000,-1.0,1.000000,1.000000,1.000000,-1.0,-0.008403,...,0.071038,0.033333,0.402208,-0.326239,0.561256,0.152542,0.035052,0.845361,-0.103448,0.719593
4,1.000000e+01,2036-04-11 00:00:00+00:00,14,-1.000000,-1.0,0.866667,0.933333,0.866667,-1.0,-0.008403,...,0.071038,0.033333,0.424837,-0.256200,0.616880,0.152542,0.022680,0.793814,-0.103448,0.691287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5256,4.771510e+16,2036-05-05 00:00:00+00:00,22,1.000000,-0.5,-1.000000,-1.000000,-1.000000,-0.5,1.000000,...,0.081921,0.033333,0.450980,-0.280081,0.578947,0.152542,-0.057642,0.817107,-0.103448,0.701828
5257,4.771510e+16,2036-05-05 00:00:00+00:00,23,1.000000,0.0,-1.000000,-1.000000,-1.000000,0.0,0.882353,...,0.065597,0.377778,0.450980,-0.280081,0.578947,0.593220,-0.060324,0.878704,0.482759,0.736094
5258,4.771510e+16,2036-05-06 00:00:00+00:00,0,1.000000,-1.0,-1.000000,-1.000000,-1.000000,-0.5,0.394958,...,0.076480,0.033333,0.450980,-0.280081,0.578947,0.152542,-0.685207,0.859455,-0.103448,0.560111
5259,4.771510e+16,2036-05-06 00:00:00+00:00,1,1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.470588,...,0.065597,0.033333,0.450980,-0.280081,0.578947,0.152542,-0.081440,0.828657,-0.103448,0.423381


In [7]:
#minmax scale
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(data_loader.mid_term[["count"]])
a = scaler.transform(data_loader.mid_term[["count"]])
b = scaler.fit_transform(data_loader.mid_term[["count"]])


In [8]:
a

array([[-1.        , -1.        , -1.        , ..., -0.76470588,
        -1.        , -0.07563025],
       [-1.        , -1.        , -0.36666667, ..., -0.00840336,
        -1.        , -0.00840336],
       [-1.        , -1.        ,  0.95      , ..., -0.00840336,
        -1.        , -0.00840336],
       ...,
       [ 1.        , -1.        , -1.        , ...,  1.        ,
        -1.        , -0.21008403],
       [ 1.        , -1.        , -1.        , ...,  1.        ,
        -1.        , -0.05882353],
       [-0.28333333, -1.        , -1.        , ..., -0.29411765,
        -1.        , -0.8487395 ]])

In [9]:
b

array([[-1.        , -1.        , -1.        , ..., -0.76470588,
        -1.        , -0.07563025],
       [-1.        , -1.        , -0.36666667, ..., -0.00840336,
        -1.        , -0.00840336],
       [-1.        , -1.        ,  0.95      , ..., -0.00840336,
        -1.        , -0.00840336],
       ...,
       [ 1.        , -1.        , -1.        , ...,  1.        ,
        -1.        , -0.21008403],
       [ 1.        , -1.        , -1.        , ...,  1.        ,
        -1.        , -0.05882353],
       [-0.28333333, -1.        , -1.        , ..., -0.29411765,
        -1.        , -0.8487395 ]])

In [12]:
data_loader.train_x[34][:,12]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
pd.DataFrame(data_loader.train_x[34])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,38,1,0,1,120,120,120,1,240,120,1,120,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,39,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,40,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,41,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,42,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,43,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,44,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,45,1,0,0,120,120,120,0,240,120,0,120,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,46,1,0,1,119,119,119,1,228,109,1,119,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,1,1,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,47,1,0,0,60,96,60,0,240,120,0,120,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
data.iloc[:,3:]

,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,-0.6,1.000000,1.000000,1.000000,-0.6,1.000000,1.000000,-0.6,1.000000,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-0.333333,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-0.333333,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-1.000000,1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-1.000000,1.0,-1.0,1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.000000,1.0,-1.0,1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-1.000000,1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-0.333333,1.0,-1.0,1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-1.0,1.000000,1.000000,1.000000,-1.0,1.000000,1.000000,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,-0.333333,1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.000000,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,-0.6,0.983333,0.983333,0.983333,-0.6,0.900000,0.816667,-0.6,0.983333,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,-0.333333,1.0,-1.0,1.0,0.0,1.0,1.0,-1.0,-1.0,1.0,-1.000000,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.

In [23]:
from docker.src.model import Autoencoder
from keras.callbacks import EarlyStopping, ModelCheckpoint

train_measure, valid_measure = train_test_split(measurement_df,
                                                      train_size=(1 - valid_size),
                                                      test_size=valid_size,
                                                      random_state=data_split_random_seed)

autoen = Autoencoder(train_measure.iloc[:,3:])

callbacks = [
ModelCheckpoint(filepath=os.path.join(task_path, 'encoder-{epoch:02d}-{val_loss:2f}.hdf5'),
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=False,
            verbose=True
),
EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=2, mode='auto')
]

autoen.train(train_measure.iloc[:,3:],
             valid_measure.iloc[:,3:], 
             epochs = 10, 
             batch_size = int(np.floor(len(train_measure.iloc[:,3:]))),
             verbose = 2,
            callbacks = callbacks)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 108)               0         
_________________________________________________________________
encoder1 (Dense)             (None, 128)               13952     
_________________________________________________________________
encoder2 (Dense)             (None, 64)                8256      
_________________________________________________________________
decoder1 (Dense)             (None, 108)               7020      
Total params: 29,228
Trainable params: 29,228
Non-trainable params: 0
_________________________________________________________________
None
Train on 1824 samples, validate on 457 samples
Epoch 1/10
 - 0s - loss: 726.2963 - val_loss: 717.0902

Epoch 00001: val_loss improved from inf to 717.09021, saving model to ./data/volume/local_test\encoder-01-717.090210.hdf5
Epoch 2/10
 - 0s - los

In [25]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.models import model_from_json 


model_path = tf.train.latest_checkpoint(task_path)
if model_path is None:
  file_name = sorted([file_name for file_name in os.listdir(task_path) if file_name.endswith('.hdf5') and file_name.startswith('encoder')])[-1]
  model_path = os.path.join(task_path, file_name)

model = model.load_weights(model_path)


NameError: name 'load_weights' is not defined

In [23]:
file_name

'encoder-10-725.665955.hdf5'

In [37]:
from keras.layers import Input
from keras.models import Model

train_measure, valid_measure = train_test_split(measurement_df,
                                                      train_size=(1 - valid_size),
                                                      test_size=valid_size,
                                                      random_state=data_split_random_seed)

input_img = Input(shape=(train_measure.iloc[:,3:].shape[1],))
layer1=autoen.model.layers[1]
layer2=autoen.model.layers[2]

encoder= Model(input_img, layer2(layer1(input_img)))
output=encoder.predict(train_measure.iloc[:,3:])


In [6]:
a = np.zeros([100,20,4])

In [11]:
a.shape[1:3]

(20, 4)

In [15]:
from keras.layers import GRU
from keras.optimizers import adam
from keras.callbacks import History
from keras.layers import Input, Dense, Masking, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.initializers import TruncatedNormal

input_img = Input(shape=(a.shape[1],a.shape[2],))
x = GRU(32,activation = 'tanh',
          return_sequences=True)(input_img)

output = Dense(a.shape[2], activation ='tanh')(x)

# 입력을 입력의 재구성으로 매핑할 모델
autoencoder = Model(input_img, output)


autoencoder.compile(loss='mean_squared_error',optimizer=adam(lr = 0.003))

print(autoencoder.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 20, 4)             0         
_________________________________________________________________
gru_3 (GRU)                  (None, 20, 32)            3552      
_________________________________________________________________
dense_2 (Dense)              (None, 20, 4)             132       
Total params: 3,684
Trainable params: 3,684
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
    input_img = Input(shape=(self.train_measure.shape[1],))
    layer1=self.model.layers[1]
    layer2=self.model.layers[2]

    encoder= Model(input_img, layer2(layer1(input_img)))
    output=encoder.predict(total_measure)
    return output

NameError: name 'self' is not defined

In [26]:
def autoencoder():
    train_measure, valid_measure = train_test_split(measurement_df,
                                                          train_size=(1 - valid_size),
                                                          test_size=valid_size,
                                                          random_state=data_split_random_seed)
    autoen = Autoencoder(train_measure.iloc[:,3:])
    autoen.train(train_measure.iloc[:,3:],
             valid_measure.iloc[:,3:], 
             epochs = 10, 
             batch_size = int(np.floor(len(train_measure.iloc[:,3:]))),
             verbose = 2,
            callbacks = [])
    self.model = autoen
    output = autoen.predict(measurement_df.iloc[:,3:])
    mesurement_df[:,3:] = output
    return measurement_df

In [ ]:
autoen.train(train_measure.iloc[:,3:],
             valid_measure.iloc[:,3:], 
             epochs = 10, 
             batch_size = int(np.floor(len(train_measure.iloc[:,3:]))),
             verbose = 2,
            callbacks = [])

output = autoen.predict(measurement_df.iloc[:,3:])

#preprocessing

In [5]:
from docker.src.data_loader import DataLoader
data_path = data_path='./data'
data_loader = DataLoader(data_path=os.path.join(data_path, 'train'),
                         common_path=os.path.join(data_path, 'volume'),
                         is_train = True,
                         task_path=task_path)

data_loader extract_outcome_cohort time: 0.3929615020751953
data_loader extract_person time: 0.025004863739013672
data_loader extract_condition time: 0.031996965408325195
data_loader extract_measurement time: 2.231036424636841
data_loader groupby_hour_condition time: 0.010988235473632812
condition_shape :  (55, 15)
there is Na after interpolation
data_loader groupby_hour_measurement time: 6.492940425872803
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 108)               0         
_________________________________________________________________
encoder1 (Dense)             (None, 128)               13952     
_________________________________________________________________
encoder2 (Dense)             (None, 128)               16512     
_________________________________________________________________
decoder1 (Dense)             (None, 108)     

In [21]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from docker.src.data_loader import DataLoader
from docker.src.model import SimpleRNNModel
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import f1_score, roc_auc_score
import datetime

log_path = os.path.join(data_path, 'volume', 'logs')

task_log_path = os.path.join(log_path, 'local_test')

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_loader, fraction, repeat):
        'Initialization'
        self.xt, self.yt,self.nx, self.ny = data_loader()
        self.fraction = fraction
        self.repeat = repeat
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.nx) / 5))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
            #positive_valid patient중 negative data undersampling    
        rand_false2 = np.random.choice(self.nx.shape[0], size=int(np.floor(self.nx.shape[0]*self.fraction)))
        random_nx = self.nx[rand_false2]
        random_ny = self.ny[rand_false2]
        
        xt = np.repeat(self.xt, self.repeat, axis=0)
        yt = np.repeat(self.yt, self.repeat, axis=0)
        
        train_x = np.concatenate([xt,random_nx], axis=0)
        train_y = np.concatenate([yt,random_ny], axis=0)
            
        if len(train_x) == len(train_y):
            p = np.random.permutation(len(train_x))
            train_x = train_x[p]
            self.train_y = train_y[p]  
        else:
            print("there is non match")
        self.train_x = pad_sequences(train_x, padding='post', value=-5)
        return (self.train_x, self.train_y)   
    
    def shape(self):
        return self.train_x.shape
    
callbacks = [
    ModelCheckpoint(filepath=os.path.join(task_path, 'model-{epoch:02d}-{val_loss:.2f}.hdf5'),
                    monitor='val_loss',
                    mode='min',
                    save_best_only=True,
                    save_weights_only=False,
                    verbose=True
    ),
    TensorBoard(log_dir=task_log_path,
                write_graph=True
    ),
     EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=2, mode='auto')

]
 # data generation 
traingen = DataGenerator(data_loader.get_train_data,fraction = 0.1, repeat = 5)
valid_gen = DataGenerator(data_loader.get_valid_data,fraction = 0.1, repeat = 5)

In [22]:
sample_x,sample_y = traingen.__getitem__(1)
print("sample_x shape", sample_x.shape)
print("sample_y positive percents", sample_y.sum()/len(sample_y))

print("time before model train",datetime.datetime.now())
model = SimpleRNNModel(shape=sample_x.shape[2])
del sample_x #memory save 

# model train 

model.train(traingen, valid_gen, epochs=10, valid_steps = 10, 
            step_epoch = 10, verbose=2, callbacks=callbacks, workers=-1)


sample_x shape (320, 100, 142)
sample_y positive percents 0.765625
time before model train 2020-01-04 01:44:13.812893
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, 142)         0         
_________________________________________________________________
masking_2 (Masking)          (None, None, 142)         0         
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                16800     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total p

KeyboardInterrupt: 